# Approach3
 - For Feedback prize-2021 competition, approach 2 will be divide each content into batch of 512, in that way we won't need to trim the content till 512 .
 - https://towardsdatascience.com/how-to-apply-transformers-to-any-length-of-text-a5601410af7f - Followed this blog fir this approach.
 - https://towardsdatascience.com/advanced-techniques-for-fine-tuning-transformers-82e4e61e16e#6196 - Use Layer wise learning rate . 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


## **Create Dataset for Transformer model**
- Inorder to create Training data, we need to create divide each content, into chunks of 510 tokens. Bert Transformer takes max_size 512



In [2]:
label_df_new=pd.read_csv("../input/feedback-nb1/train_file_labels2.csv")
label_df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15594 entries, 0 to 15593
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         15594 non-null  object
 1   text       15594 non-null  object
 2   label_str  15594 non-null  object
dtypes: object(3)
memory usage: 365.6+ KB


In [3]:
# let's find out the number of unique tags and their count
from collections import Counter

labels_lst = []
for label_str in label_df_new["label_str"]:
    labels_lst.extend(label_str.strip().split(","))
labels_count = Counter(labels_lst)
labels_count = {k: v for k, v in sorted(labels_count.items(), key=lambda item: item[1], reverse=True)}
print(len(labels_count))
labels_count

15


{'I-Evidence': 3489544,
 'I-Claim': 824107,
 'I-Concluding Statement': 814366,
 'I-Lead': 474070,
 'O': 305458,
 'I-Position': 265856,
 'I-Counterclaim': 133965,
 'I-Rebuttal': 117445,
 'B-Claim': 50202,
 'B-Evidence': 45702,
 'B-Position': 15419,
 'B-Concluding Statement': 13505,
 'B-Lead': 9305,
 'B-Counterclaim': 5817,
 'B-Rebuttal': 4337}

In [4]:
!!pip install transformers seqeval[gpu]

['Requirement already satisfied: transformers in /opt/conda/lib/python3.7/site-packages (4.16.2)',
 'Collecting seqeval[gpu]',
 '  Downloading seqeval-1.2.2.tar.gz (43 kB)',
 '  Preparing metadata (setup.py): started',
 "  Preparing metadata (setup.py): finished with status 'done'",
 'Requirement already satisfied: huggingface-hub<1.0,>=0.1.0 in /opt/conda/lib/python3.7/site-packages (from transformers) (0.4.0)',
 'Requirement already satisfied: pyyaml>=5.1 in /opt/conda/lib/python3.7/site-packages (from transformers) (6.0)',
 'Requirement already satisfied: importlib-metadata in /opt/conda/lib/python3.7/site-packages (from transformers) (4.11.3)',
 'Requirement already satisfied: requests in /opt/conda/lib/python3.7/site-packages (from transformers) (2.26.0)',
 'Requirement already satisfied: tokenizers!=0.11.3,>=0.10.1 in /opt/conda/lib/python3.7/site-packages (from transformers) (0.11.6)',
 'Requirement already satisfied: numpy>=1.17 in /opt/conda/lib/python3.7/site-packages (from t

In [5]:
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
import transformers
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification

In [6]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [7]:
import pprint as pprint
label_to_idx = {'B-Claim': 0, 'I-Claim': 1,
                'B-Evidence': 2, 'I-Evidence': 3,
                'B-Position': 4, 'I-Position': 5,
                'B-Concluding Statement': 6, 'I-Concluding Statement': 7,
                'B-Lead': 8, 'I-Lead': 9,
                'B-Counterclaim': 10, 'I-Counterclaim': 11,
                'B-Rebuttal': 12, 'I-Rebuttal': 13,
                'O': 14}
idx_to_label = {v:k for k,v in label_to_idx.items()}
idx_to_label

{0: 'B-Claim',
 1: 'I-Claim',
 2: 'B-Evidence',
 3: 'I-Evidence',
 4: 'B-Position',
 5: 'I-Position',
 6: 'B-Concluding Statement',
 7: 'I-Concluding Statement',
 8: 'B-Lead',
 9: 'I-Lead',
 10: 'B-Counterclaim',
 11: 'I-Counterclaim',
 12: 'B-Rebuttal',
 13: 'I-Rebuttal',
 14: 'O'}

In [8]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 2
VALID_BATCH_SIZE = 2
EPOCHS = 25
LEARNING_RATE = 5e-05
MAX_GRAD_NORM = 10
# tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [9]:
tokenizer.save_pretrained("./bert-base-uncased-tokenizer")

('./bert-base-uncased-tokenizer/tokenizer_config.json',
 './bert-base-uncased-tokenizer/special_tokens_map.json',
 './bert-base-uncased-tokenizer/vocab.txt',
 './bert-base-uncased-tokenizer/added_tokens.json',
 './bert-base-uncased-tokenizer/tokenizer.json')

In [10]:
# include all these functions in Dataset class
class dataset(Dataset):
  def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

  def __getitem__(self, index):
        # step 1: get the sentence and word labels 
        sentence = self.data.text[index].strip().split()  
        word_labels = self.data.label_str[index].split(",") 

        encoding = tokenizer.encode_plus(sentence,
                                         add_special_tokens=False,
                                         return_tensors="pt",
                                         is_split_into_words=True,
                                         return_offsets_mapping=True)
        
        # step 3: create token labels only for first word pieces of each tokenized word
        labels = [label_to_idx[label] for label in word_labels] 
        # code based on https://huggingface.co/transformers/custom_datasets.html#tok-ner
        # create an empty array of -100 of length max_length
        encoded_labels = np.ones(len(encoding["offset_mapping"][0]), dtype=int) * -100
        
        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        for idx, mapping in enumerate(encoding["offset_mapping"][0]):
          if mapping[0] == 0 and mapping[1] != 0:
            # overwrite label
            encoded_labels[idx] = labels[i]
            i += 1
        encoded_labels = torch.as_tensor(encoded_labels)
        
        # divide the content and encodings into chunks of MAX_LEN
        chunksize = self.max_len

        # Step4: split into chunks of 510 tokens, we also convert to list (default is tuple which is immutable)
        input_id_chunks = list(encoding['input_ids'][0].split(chunksize - 2))
        mask_chunks = list(encoding['attention_mask'][0].split(chunksize - 2))
        offset_mapping_chunks = list(encoding['offset_mapping'][0].split(chunksize - 2))
        encoded_labels_chunks = list(encoded_labels.split(chunksize - 2))
        
        
        # Step 5: loop through each chunk
        for i in range(len(input_id_chunks)):
            # add CLS and SEP tokens to input IDs
            input_id_chunks[i] = torch.cat([
                torch.tensor([101]), input_id_chunks[i], torch.tensor([102])
            ])
            # add attention tokens to attention mask
            mask_chunks[i] = torch.cat([
                torch.tensor([1]), mask_chunks[i], torch.tensor([1])
            ])

            # add offset_mapping tokens to offset_maps
            offset_mapping_chunks[i] = torch.cat([
                torch.tensor([[0,0]]), offset_mapping_chunks[i], torch.tensor([[0,0]])
            ])

            # to the encoded_labels also, add -100
            encoded_labels_chunks[i] = torch.cat([
                torch.tensor([-100]), encoded_labels_chunks[i], torch.tensor([-100])
            ])


            # get required padding length
            pad_len = chunksize - input_id_chunks[i].shape[0]
            # check if tensor length satisfies required chunk size
            if pad_len > 0:
                # if padding length is more than 0, we must add padding
                input_id_chunks[i] = torch.cat([
                    input_id_chunks[i], torch.Tensor([0] * pad_len)
                ])
                mask_chunks[i] = torch.cat([
                    mask_chunks[i], torch.Tensor([0] * pad_len)
                ])
                offset_mapping_chunks[i] = torch.cat([
                    offset_mapping_chunks[i], torch.Tensor([[0,0]] * pad_len)
                ])
                encoded_labels_chunks[i] = torch.cat([
                    encoded_labels_chunks[i], torch.Tensor([-100] * pad_len)
                ])

        # Step 6: Stack the chunks
        input_ids = torch.stack(input_id_chunks)
        attention_mask = torch.stack(mask_chunks)
        offset_mapping_merged = torch.stack(offset_mapping_chunks)
        encoded_labels_merged = torch.stack(encoded_labels_chunks)

        # step 7: turn everything into PyTorch tensors
        input_dict = {
            'input_ids': input_ids.long(),
            'attention_mask': attention_mask.int(),
            'offset_mapping': offset_mapping_merged,
            'labels': encoded_labels_merged.long()
        }
        
#         item = {key: torch.as_tensor(val) for key, val in encoding.items()}
#         item['labels'] = torch.as_tensor(encoded_labels)
        
        return input_dict

  def __len__(self):
        return self.len

In [11]:
train_size = 0.8
train_dataset = label_df_new.sample(frac=train_size,random_state=200)
test_dataset = label_df_new.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

In [12]:
training_set = dataset(train_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)

In [13]:
def custom_collate_fn(data_list):
    # convert each key into a list of keys
    data_key_list = {key: [data[key] for data in data_list] for key in data_list[0]}
#     input_ids_list = [data['input_ids'] for data in data_list]
#     attention_mask_list = [data['attention_mask'] for data in data_list]
#     offset_mapping_list = [data['offset_mapping'] for data in data_list]
#     labels_list = [data['labels'] for data in data_list]
    
    # concat all the key on dim=0
    # i.e if one example has (1,512) and 2nd example has (2,512)
    # this may be able to handle
    data_key_cat = {key: torch.cat(data_key_list[key], dim=0) for key in data_key_list}
#     input_ids_cat = torch.cat(input_ids_list, dim=0)
#     attention_mask_cat = torch.cat(attention_mask_list, dim=0)
#     offset_mapping_cat = torch.cat(offset_mapping_list, dim=0)
#     labels_cat = torch.cat(labels_list, dim=0)

    return data_key_cat

In [14]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params, collate_fn=custom_collate_fn)
testing_loader = DataLoader(testing_set, **test_params, collate_fn=custom_collate_fn)

## **Load the Model**

In [15]:
model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(label_to_idx))
model.to(device)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [16]:
## sanity check
inputs = training_set[1]
input_ids = inputs["input_ids"].unsqueeze(0)
attention_mask = inputs["attention_mask"].unsqueeze(0)
labels = inputs["labels"].unsqueeze(0)

input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

outputs = model(input_ids[0], attention_mask=attention_mask[0], labels=labels[0])
initial_loss = outputs[0]
initial_loss

Token indices sequence length is longer than the specified maximum sequence length for this model (790 > 512). Running this sequence through the model will result in indexing errors


tensor(2.8092, device='cuda:0', grad_fn=<NllLossBackward>)

## **Learning Rate for different layers**
  - Tried to implement this but facing issues
      - ValueError: some parameters appear in more than one parameter group
  - Approach 2 - Use Learning rate scheduler with warm-up steps instead. Same Learning rate for all layers.

In [17]:
opt_parameters = []    # To be passed to the optimizer (only parameters of the layers you want to update).
named_parameters = list(model.named_parameters()) 

# According to AAAMLP book by A. Thakur, we generally do not use any decay 
# for bias and LayerNorm.weight layers.
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
init_lr = 3.5e-6 
head_lr = 3.6e-6
lr = init_lr

In [18]:
# for 12 hidden layers
for layer in range(11, -1, -1):
    params_0 = [model.get_parameter(n) for n,p in named_parameters if f"bert.encoder.layer.{layer}" in n
               and any(nd in n for nd in no_decay)]
    params_1 = [model.get_parameter(n) for n,p in named_parameters if f"bert.encoder.layer.{layer}" in n
               and not any(nd in n for nd in no_decay)]
    layer_params = {"params": params_0, "lr": lr, "weight_decay": 0.0}
    opt_parameters.append(layer_params)
    
    layer_params = {"params": params_1, "lr": lr, "weight_decay": 0.01}
    opt_parameters.append(layer_params)
    
    lr *= 0.9

# === Embeddings layer ==========================================================
params_0 = [model.get_parameter(n) for n,p in named_parameters if "embeddings" in n
            and any(nd in n for nd in no_decay)]
params_1 = [model.get_parameter(n) for n,p in named_parameters if "embeddings" in n
            and not any(nd in n for nd in no_decay)]

embed_params = {"params": params_0, "lr": lr, "weight_decay": 0.0} 
opt_parameters.append(embed_params)
        
embed_params = {"params": params_1, "lr": lr, "weight_decay": 0.01} 
opt_parameters.append(embed_params)  

In [19]:
# optimizer = transformers.AdamW(opt_parameters, lr=init_lr)

### Learning Rate Scheduling with Warm_up_steps

In [20]:
# optimizer = torch.optim.Adam(opt_parameters, lr=init_lr)
# optimizer = transformers.AdamW(opt_parameters, lr=init_lr)
num_training_steps = EPOCHS * len(training_loader)
optimizer = transformers.AdamW(model.parameters(), lr=5e-5)
lr_scheduler = transformers.get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=50,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

155950


/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [21]:
#efining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(training_loader):
        
        # after using custom collate_fn in DataLoader, need to update the same
        ids = batch['input_ids'].unsqueeze(0)
        mask = batch['attention_mask'].unsqueeze(0)
        labels = batch['labels'].unsqueeze(0)
        
        ids = ids.to(device, dtype = torch.long)
        mask = mask.to(device, dtype = torch.long)
        labels = labels.to(device, dtype = torch.long)

        output = model(input_ids=ids[0], attention_mask=mask[0], labels=labels[0])
        loss = output[0]
        tr_logits = output[1]
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels[0].size(0)
        
        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = labels[0].view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        # if flattened_target is [1024,] i.e [2*512,] where 2 is batch_size and 512 is seq_length
        # and out of 1024 i.e 50 tokens have -100 i.e they don't have a label assigned then
        # labels is [974,] and predictions is [974,]
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [22]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 100 training steps: 2.744267463684082
Training loss per 100 training steps: 1.6253047402542415
Training loss per 100 training steps: 1.4794797138195133
Training loss per 100 training steps: 1.3954441541453138
Training loss per 100 training steps: 1.325261650834595
Training loss per 100 training steps: 1.2831850871592463
Training loss per 100 training steps: 1.2425245122485074
Training loss per 100 training steps: 1.208709621097834
Training loss per 100 training steps: 1.1802214182048254
Training loss per 100 training steps: 1.1592794021676305
Training loss per 100 training steps: 1.1407017129939514
Training loss per 100 training steps: 1.1282861508043325
Training loss per 100 training steps: 1.1110868513286363
Training loss per 100 training steps: 1.097030442188008
Training loss per 100 training steps: 1.0852865195972081
Training loss per 100 training steps: 1.073166509952488
Training loss per 100 training steps: 1.0613324443561296
Training loss per 

In [23]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            # after using custom collate_fn in DataLoader, need to update the same
            ids = batch['input_ids'].unsqueeze(0)
            mask = batch['attention_mask'].unsqueeze(0)
            labels = batch['labels'].unsqueeze(0)
            
            ids = ids.to(device, dtype = torch.long)
            mask = mask.to(device, dtype = torch.long)
            labels = labels.to(device, dtype = torch.long)
            
            output = model(input_ids=ids[0], attention_mask=mask[0], labels=labels[0])
            loss = output[0]
            eval_logits = output[1]
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
#             eval_labels.append(labels)
#             eval_preds.append(predictions)
            
            eval_labels.append([idx_to_label[id.item()] for id in labels])
            eval_preds.append([idx_to_label[id.item()] for id in predictions])
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

#     labels = [idx_to_label[id.item()] for id in eval_labels]
#     predictions = [idx_to_label[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

#     return labels, predictions
    return eval_labels, eval_preds

In [24]:
labels, predictions = valid(model, testing_loader)

Validation loss per 100 evaluation steps: 1.6496251821517944
Validation loss per 100 evaluation steps: 1.677530144995982
Validation loss per 100 evaluation steps: 1.8091944140284808
Validation loss per 100 evaluation steps: 1.7915710346445293
Validation loss per 100 evaluation steps: 1.8071055493301287
Validation loss per 100 evaluation steps: 1.843200724579379
Validation loss per 100 evaluation steps: 1.8507944083650179
Validation loss per 100 evaluation steps: 1.8372127332973072
Validation loss per 100 evaluation steps: 1.8490664685672886
Validation loss per 100 evaluation steps: 1.8467704485212924
Validation loss per 100 evaluation steps: 1.8401574868035244
Validation loss per 100 evaluation steps: 1.8429577501928882
Validation loss per 100 evaluation steps: 1.8436938303396366
Validation loss per 100 evaluation steps: 1.8402466138081768
Validation loss per 100 evaluation steps: 1.8295785128900104
Validation loss per 100 evaluation steps: 1.8260407674012782
Validation Loss: 1.8236450

In [25]:
from seqeval.metrics import classification_report

print(classification_report(labels, predictions))

                      precision    recall  f1-score   support

               Claim       0.30      0.40      0.34      9890
Concluding Statement       0.32      0.49      0.39      2679
        Counterclaim       0.19      0.27      0.22      1168
            Evidence       0.17      0.24      0.20      9059
                Lead       0.42      0.54      0.47      1837
            Position       0.37      0.48      0.42      3080
            Rebuttal       0.12      0.18      0.14       866

           micro avg       0.26      0.36      0.30     28579
           macro avg       0.27      0.37      0.31     28579
        weighted avg       0.26      0.36      0.31     28579



In [26]:
import os
dirname = "/kaggle/working/feedback-bert-uncased-model3/"
if not os.path.isdir(dirname):
    os.makedirs(dirname)
    
model.save_pretrained(dirname)